In [ ]:
%cd image_retrieval/


/home/anlab/Downloads/imageclassification/image_retrieval


In [ ]:
path_local="/home/anlab/Downloads/imageclassification/"


***Lấy file query ***

In [ ]:

import os
def get_file_query():
  path= "gt_files/"
  list_gt = os.listdir(path)
  list_files_query=[]
  for file_name in list_gt:
    name=file_name.split(".")[0].split("_")[-1]
    if(name=="query"):
      list_files_query.append(file_name)
  return list_files_query








**lấy file ảnh tương ứng với querry (gồm 3 file good, oke, junk)**

In [ ]:
#get file good,oke,junk correspond
def get_file_name(list_files_query, name):
  list_files_name=[]
  for query_file in list_files_query:
    name_correspond= query_file.replace("query",name)
    list_files_name.append(name_correspond)
  return list_files_name
  




**lấy ảnh trong file query**

In [ ]:
def get_image_query(list_files_query):
  list_image_query=[]
  for query in list_files_query:
    dir= path_local+"/image_retrieval/gt_files/"+query
    with open(dir,"r") as f:
      for line in f.readlines():
        name_image= line.split(" ")[0].split("_",1)[1]+".jpg"
        list_image_query.append(name_image)
  return list_image_query

**lấy ảnh tương ứng của query với từng file good, oke, junk**

In [ ]:


def get_image_correspond(file_name):
  list_image_name= []
  with open("gt_files/"+file_name,"r") as f:
    for line in f.readlines():
      list_image_name.append(line.split("\n")[0]+".jpg")
  return list_image_name

#Rmac


In [ ]:
%cd keras_rmac/


/home/anlab/Downloads/imageclassification/image_retrieval/keras_rmac


In [ ]:
from __future__ import division
from __future__ import print_function
from get_regions import rmac_regions, get_size_vgg_feat_map
from PIL import Image,ImageOps
import scipy.io
import numpy as np
import utils
from torchvision import models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch
import cv2

def rzac(x, L=3, eps=1e-6):
#     N, C, H, W = x.size()
    feature_total= []
    ovr = 0.4 # desired overlap of neighboring regions
    steps = torch.Tensor([2, 3, 4, 5, 6, 7]) # possible regions for the long dimension

    W = x.size(3)
    H = x.size(2)

    w = min(W, H)
    w2 = math.floor(w/2.0 - 1)

    b = (max(H, W)-w)/(steps-1)
    (tmp, idx) = torch.min(torch.abs(((w**2 - w*b)/w**2)-ovr), 0) # steps(idx) regions for long dimension

    # region overplus per dimension
    Wd = 0;
    Hd = 0;
    if H < W:  
        Wd = idx.item() + 1
    elif H > W:
        Hd = idx.item() + 1

    v = F.max_pool2d(x, (x.size(-2), x.size(-1)))
    v = v / (torch.norm(v, p=2, dim=1, keepdim=True) + eps).expand_as(v)
    
    for l in range(1, L+1):
        
        wl = math.floor(2*w/(l+1))
        wl2 = math.floor(wl/2 - 1)
        
        if l+Wd == 1:
            b = 0
        else:
            b = (W-wl)/(l+Wd-1)
        cenW = torch.floor(wl2 + torch.Tensor(range(l-1+Wd+1))*b) - wl2 # center coordinates
        if l+Hd == 1:
            b = 0
        else:
            b = (H-wl)/(l+Hd-1)
        cenH = torch.floor(wl2 + torch.Tensor(range(l-1+Hd+1))*b) - wl2 # center coordinates
        
        for i_ in cenH.tolist():
            for j_ in cenW.tolist():
                if wl == 0:
                    continue
                R = x[:,:,(int(i_)+torch.Tensor(range(wl)).long()).tolist(),:]
                R = R[:,:,:,(int(j_)+torch.Tensor(range(wl)).long()).tolist()]
                vt = F.max_pool2d(R, (R.size(-2), R.size(-1)))
                vt = vt / (torch.norm(vt, p=2, dim=1, keepdim=True) + eps).expand_as(vt)
                v += vt
                feature_total.append(vt)
    feature_total=torch.vstack(feature_total)
    feature_total=torch.squeeze(feature_total,axis=2)
    return feature_total

def get_feature(reg_feat_mat):
  reg_feat_mat = reg_feat_mat.squeeze() # [1, r, c] -> [r, c]
  r = reg_feat_mat.size(0)
  ag_feat_vec = torch.sum(reg_feat_mat, dim=0, keepdim=False) # (r, c) -> (c,)
  ag_feat_vec = ag_feat_vec / (torch.norm(ag_feat_vec,
                                            p=2,
                                            dim=0,
                                            keepdim=True) + 1e-6)
#   ag_feat_vec=np.array(ag_feat_vec).reshape(1,-1)
  ag_feat_vec= torch.reshape(ag_feat_vec,(1,-1))
  return ag_feat_vec

In [ ]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tqdm
def load_image_and_bbs(im_path, bb_path):
    img = cv2.imread(im_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    bb_df = pd.read_csv(bb_path, sep=' ', header=None, index_col=False)
    bb_mat = bb_df.to_numpy()
    return img, bb_mat

def _get_masked_img( img, bbs):
    patches = []
    masked = np.zeros_like(img)
    for bb in bbs:

        x_l = bb[0]
        x_r = bb[0] + bb[2]
        y_u = bb[1]
        y_d = bb[1] + bb[3]
        patches.append(img[y_u:y_d, x_l:x_r])
        masked[y_u:y_d, x_l:x_r] = img[y_u:y_d, x_l:x_r]

    return masked

In [ ]:
import os
list_files_query=os.listdir(path_local+"data/pg_data/query")
from shutil import copyfile
path=path_local+"data/pg_data/"
for i in range(len(list_files_query)):
  name_file=list_files_query[i]
  with open(path+"query/"+name_file,"r") as f:
    for line in f.readlines():
      line= line.split(" ",1)
      name_images=line[0].split("_",1)[1]+".jpg"
      coordinate= line[1].split("\n")[0].split(" ")
      coordinate=[round(float(value)) for value in coordinate]
      coordinate=str(coordinate[0])+" "+str(coordinate[1])+" "+str(coordinate[2]-coordinate[0])+" "+str(coordinate[3]-coordinate[1])
      if(i+1<10):
        copyfile(path+"Images/"+name_images, path+"Queries/"+f"0{i+1}.jpg")
        with open(path+"Queries/"+f"0{i+1}.txt","w") as f:
          f.write(coordinate)
      else :
        copyfile(path+"Images/"+name_images, path+"Queries/"+f"{i+1}.jpg")
        with open(path+"Queries/"+f"{i+1}.txt","w") as f:
          f.write(coordinate)
      

In [ ]:
import os
query_dir = path_local+'data/pg_data/Queries'
im_bb_path_tuples = [(os.path.join(query_dir, '{:02d}.jpg'.format(i)),
                      os.path.join(query_dir, '{:02d}.txt'.format(i))
                     ) for i in range(1, 56)
                    ]
len(im_bb_path_tuples)

55

Tạo model


In [ ]:
from torchvision import models
import torch
device = 'cuda'
backbone= models.resnet101(pretrained=True)
model=torch.nn.Sequential(*list(backbone.children())[:-2])
model.to('cuda')
model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [ ]:
def change_chanel(tensor_img):
    x= np.moveaxis(tensor_img, -1, 0)
    x=np.expand_dims(x, axis=0)
    return x
def preprocessImage(linkImage):
    img=cv2.imread(linkImage)
    x = change_chanel(img)
    x=torch.tensor(x)
    x = x.float().to('cuda')
    return x

In [ ]:
torch.cuda.is_available()

True

In [ ]:
import math
from tqdm import tqdm
import tensorflow as tf
import keras
vector_query =[]
for value in tqdm(im_bb_path_tuples):
    demo_im_path, demo_bb_path = value
    demo_im, demo_bb = load_image_and_bbs(demo_im_path, demo_bb_path)
    masked=_get_masked_img(demo_im, demo_bb)
    img=masked
    x=change_chanel(img)
#     print(img.shape)
    x=torch.tensor(x)
    x = x.float().to('cuda')
    feature = model(x)
    feature=torch.tensor(feature)
    #caculate rzac
    reg_feat_mat= rzac(feature)
    #caculate rmac 
    RMAC = get_feature(reg_feat_mat)
    vector_query.append(RMAC[-1])
print(len(vector_query))

  0%|                                                    | 0/55 [00:00<?, ?it/s]/home/anlab/anaconda3/envs/DB/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  app.launch_new_instance()
100%|███████████████████████████████████████████| 55/55 [00:14<00:00,  3.74it/s]

55


In [ ]:
vector_query[0].shape

torch.Size([2048])

In [ ]:
def get_Rmac(file,model):
    x = preprocessImage(file)
    
    im_tensor = model(x)
    reg_feat_mat= rzac(im_tensor)
    #caculate rmac 
    RMAC = get_feature(reg_feat_mat)
    return (RMAC)


In [ ]:
%cd ..

/home/anlab/Downloads/imageclassification/image_retrieval


In [ ]:
%cd keras_rmac

/home/anlab/Downloads/imageclassification/image_retrieval/keras_rmac


In [ ]:
# get file image correspond with query
im_list_query=get_image_query(list_files_query)
print(len(im_list_query))

55


In [ ]:
import math
from tqdm import tqdm
import pickle
with open('dataset.pickle',"rb") as f:
    dataset = pickle.load(f)
vector_dataset=[]
for img in tqdm(dataset):
    file = path_local+"Object Dataset/test/oxbuild_images/"+img
    img=cv2.imread(file)
    x = change_chanel(img)
    x=torch.tensor(x)
    x = x.float().to('cuda')
    feature = model(x)
    RMAC = rzac(feature)
    vector_dataset.append(RMAC[-1])

  0%|                                          | 1/5063 [00:00<18:55,  4.46it/s]


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 3.95 GiB total capacity; 2.97 GiB already allocated; 9.94 MiB free; 3.01 GiB reserved in total by PyTorch)

In [ ]:
# vector_query=torch.stack(vector_query)
# vector_query=torch.squeeze(vector_query)
# vector_query=vector_query.numpy()
vector_query.shape

torch.Size([55, 2048])

In [ ]:
vector_query=vector_query.detach().cpu().numpy()
vector_query.shape

(55, 2048)

In [ ]:
vector_dataset=torch.stack(vector_dataset)
vector_dataset=torch.squeeze(vector_dataset)
# vector_query=vector_query.numpy()
vector_dataset.shape

torch.Size([5063, 2048])

In [ ]:
vector_dataset=vector_dataset.detach().cpu().numpy()
vector_dataset.shape

(5063, 2048)

In [ ]:
# vector_query_test=[t.numpy() for t in vector_query]
# vector_dataset_test=[t.numpy() for t in vector_dataset]
vector_query=np.array(vector_query)
vector_dataset=np.array(vector_dataset)
print(vector_query.shape)
print(vector_dataset.shape)

(55, 512)
(5063, 512)


In [ ]:
# tính giao của 2 array-> kết quả trả về theo thứ tự của array 1
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [ ]:
%cd /home/anlab/Downloads/imageclassification/image_retrieval/

/home/anlab/Downloads/imageclassification/image_retrieval


In [ ]:
# %cd ..

In [ ]:
list_files_good=get_file_name(list_files_query,"good")

In [ ]:
topk=5063


In [ ]:
def map_name(list_files_query,name,vector_query,vector_dataset,dataset):
  list_ap_name=[]
  list_files_name=get_file_name(list_files_query,name)
  for i in range(len(list_files_query)):
    vector_l2_dataset=np.linalg.norm(vector_query[i]-vector_dataset,axis=-1)
    gt=get_image_correspond(list_files_name[i])
    toplen_image_query= np.argsort(vector_l2_dataset) 
    all_image_in_top = toplen_image_query[:topk]
    image_correspond = [dataset[image] for image in all_image_in_top]
    index=[]
    AP=0
    for j in range(len(image_correspond)):
      if(image_correspond[j] in gt):
        index.append(j)
    for k in range(len(index)):
      AP= AP+ ((k+1)/(index[k]+1))
    AP=AP/len(gt)
    list_ap_name.append(AP)
  map_name=sum(list_ap_name)/len(list_ap_name)
  return map_name

In [ ]:
map_good= map_name(list_files_query,"good",vector_query,vector_dataset,dataset)

In [ ]:
print(map_good)


0.0059823119713225525
